In [5]:
import pandas as pd
from datetime import datetime, timedelta

def read_input_csv(file_path):
    """Reads the input CSV file and returns a DataFrame."""
    return pd.read_csv(file_path)

def identify_and_convert_date_columns(df):
    """Identifies date columns and converts them to datetime."""
    date_columns = [col for col in df.columns if 'date' in col.lower()]
    for col in date_columns:
        df[col] = pd.to_datetime(df[col], errors='coerce')
    return date_columns

def calculate_tenure(row):
    """Calculates tenure in years, rounded to the nearest whole number."""
    date_of_exit = row['Date of Exit'] if pd.notnull(row['Date of Exit']) else pd.Timestamp(datetime.now())
    tenure_years = (date_of_exit - row['Date of Joining']).days / 365.25
    return round(tenure_years)

def expand_records(df, date_columns):
    """Includes 'Tenure in Org' calculation and adds 'Variable Pay' column with a default value of 0."""
    all_records = []

    for _, row in df.iterrows():
        dates = sorted({row[col] for col in date_columns if pd.notnull(row[col])})

        record = {'Employee Code': row['Employee Code'], 'Effective Date': row['Date of Joining']}
        for col in df.columns:
            if 'date' not in col.lower() and col != 'Employee Code':
                record[col.split(' ')[0]] = row[col]

        # Add 'Tenure in Org' calculation
        record['Tenure in Org'] = calculate_tenure(row)
        # Initialize 'Variable Pay' to 0
        record['Variable Pay'] = 0

        all_records.append(record.copy())

        for date in dates:
            new_record = record.copy()
            for col in ['Compensation 1', 'Compensation 2', 'Review 1', 'Review 2', 'Engagement 1', 'Engagement 2']:
                date_col = f"{col} date"
                if date == row.get(date_col):
                    new_record[col.split(' ')[0]] = row[col]

            if new_record != record:
                all_records[-1]['End Date'] = date - timedelta(days=1)
                new_record['Effective Date'] = date
                all_records.append(new_record)
                record = new_record

        if all_records[-1] == record:
            all_records[-1]['End Date'] = datetime(2100, 1, 1)
        else:
            all_records.pop()

    return pd.DataFrame(all_records)

def save_transformed_data(df, output_file_path):
    """Saves the transformed DataFrame to a CSV file."""
    df.to_csv(output_file_path, index=False)

# Paths for the input and output files
input_file_path = 'input.csv'  # Adjust this path as necessary
output_file_path = 'transformed_data.csv'

# Transforming the data
input_df = read_input_csv(input_file_path)
date_columns = identify_and_convert_date_columns(input_df)
transformed_df = expand_records(input_df, date_columns)

save_transformed_data(transformed_df, output_file_path)

print("Data transformation complete. Output saved to:", output_file_path)


Data transformation complete. Output saved to: transformed_data.csv
